In [1]:
%cd ..

/home/zilian/projects/bachelorproef


/home/zilian/projects/bachelorproef/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

import cv2
import numpy as np
from src.config import FRAMES_PATH
from src.utils import iter_frames_dir
from tqdm import tqdm

In [3]:
segmentation_results = "segmentation_results"

frame_results = os.listdir(segmentation_results)
frame_results = sorted(frame_results, key=lambda x: int(x.split(".")[0]))
frame_results = {int(x.split(".")[0]): os.path.join(segmentation_results, x) for x in frame_results}

In [4]:
# Define colors for different classes (BGR format)
colors = {
    "1": (255, 0, 0),
    "2": (0, 255, 0),
    "3": (0, 0, 255),
    "4": (255, 255, 0),
    "5": (255, 0, 255),
    "6": (0, 255, 255),
    "7": (128, 0, 0),
    "8": (0, 128, 0),
    "9": (0, 0, 128),
    "10": (128, 128, 0),
    "11": (128, 0, 128),
    "12": (0, 128, 128),
}


def process_frame(args):
    """
    Process a single frame by applying segmentation masks.

    Args:
        args: tuple containing:
            - frame_idx: index of the frame
            - frame: the image frame (numpy array)
            - segmentation_results: directory path with .npz files
            - frame_result: filename for the current frame's segmentation data

    Returns:
        Tuple (frame_idx, processed_frame)
    """
    frame_idx, frame, segmentation_results, frame_result = args

    if frame_result is None:
        return frame_idx, frame

    data = np.load(frame_result)

    # Apply each segmentation mask as an overlay
    for cls in data.files:
        mask = data[cls].astype(np.uint8)[0]
        color = colors.get(cls, (255, 255, 255))
        colored_mask = np.zeros_like(frame)
        colored_mask[mask > 0] = color
        # Blend the mask with the frame
        frame = cv2.addWeighted(frame, 1.0, colored_mask, 0.5, 0)

    return frame_idx, frame


# Assume these variables are defined:
# - src_video_path: path to the source video
# - segmentation_results: directory containing the .npz files
# - frame_results: a list (or dict) mapping frame indices to segmentation filenames

# Create a video writer in the main process.
video_result = cv2.VideoWriter(
    "output.mp4",
    cv2.VideoWriter_fourcc(*"mp4v"),
    24.95,
    (1920, 1080),
)

batch_size = 200  # Adjust this size based on your memory constraints
futures = []

with ProcessPoolExecutor() as executor:
    # Loop through frames and process in batches
    for frame_idx, frame in tqdm(iter_frames_dir(FRAMES_PATH), total=len(frame_results)):
        frame_result = frame_results.get(frame_idx, None)

        futures.append(executor.submit(process_frame, (frame_idx, frame, segmentation_results, frame_result)))

        # Once we reach the batch size, process and write the batch
        if len(futures) >= batch_size:
            batch_results = {}
            for future in as_completed(futures):
                idx, processed_frame = future.result()
                batch_results[idx] = processed_frame
            # Write frames in order
            for idx in sorted(batch_results.keys()):
                video_result.write(batch_results[idx])
            # Clear the list for the next batch
            futures = []

    # Process any remaining frames that didn't fill the last batch
    if futures:
        batch_results = {}
        for future in as_completed(futures):
            idx, processed_frame = future.result()
            batch_results[idx] = processed_frame
        for idx in sorted(batch_results.keys()):
            video_result.write(batch_results[idx])

video_result.release()

  0%|          | 0/2223 [00:00<?, ?it/s]

2285it [00:33, 68.41it/s]                           
